In [37]:
# Data import
import pyomo.environ as pe
import math

# Data
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2


# possible number of long bars per long bar type per factory 
N = {
    1: {"IJmuiden": 67, "Segal": 56, "South Wales": 156},
    2: {"IJmuiden": 50, "Segal": 42, "South Wales": 117}
}

possible_longbars = {} 

for l in longbars:
    possible_longbars[l] = {} 
    for f in factories:
        possible_longbars[l][f] = list(range(1, N.get(l, {}).get(f, 0) + 1))

# Defining big M
# List to store total demand per customer
total_demand_per_customer = []

# Loop over each customer
for a in customers:
    total_demand = sum(demand[r][a][t-1] for r in rebars for t in periods)  # Sum over all rebars and periods
    total_demand_per_customer.append(total_demand)

# Get the maximum demand across all customers
M = max(total_demand_per_customer)

inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

In [ ]:
#New data for 3: 
distanceq3 = {
    "IJmuiden": {"IJmuiden": 0, "Segal": 284, "South Wales": 826, "Bochum": 250, "Boenen": 282, "Dortmund": 266, "Gelsenkirchen": 234, "Hagen": 289, "Iserlohn": 299, "Neuss": 259, "Schwerte": 279},
    "Segal": {"IJmuiden": 284, "Segal": 0, "South Wales": 750, "Bochum": 203, "Boenen": 242, "Dortmund": 222, "Gelsenkirchen": 198, "Hagen": 206, "Iserlohn": 226, "Neuss": 140, "Schwerte": 216},
    "South Wales": {"IJmuiden": 826, "Segal": 750, "South Wales": 0, "Bochum": 866, "Boenen": 914, "Dortmund": 885, "Gelsenkirchen": 859, "Hagen": 903, "Iserlohn": 913, "Neuss": 843, "Schwerte": 901},
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866, "Bochum": 0, "Boenen": 55, "Dortmund": 21, "Gelsenkirchen": 19, "Hagen": 41, "Iserlohn": 51, "Neuss": 56, "Schwerte": 39},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914, "Bochum": 55, "Boenen": 0, "Dortmund": 34, "Gelsenkirchen": 56, "Hagen": 44, "Iserlohn": 54, "Neuss": 102, "Schwerte": 32},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885, "Bochum": 21, "Boenen": 34, "Dortmund": 0, "Gelsenkirchen": 34, "Hagen": 21, "Iserlohn": 36, "Neuss": 75, "Schwerte": 15},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859, "Bochum": 19, "Boenen": 56, "Dortmund": 34, "Gelsenkirchen": 0, "Hagen": 56, "Iserlohn": 66, "Neuss": 62, "Schwerte": 54},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903, "Bochum": 41, "Boenen": 44, "Dortmund": 21, "Gelsenkirchen": 56, "Hagen": 0, "Iserlohn": 20, "Neuss": 66, "Schwerte": 14},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913, "Bochum": 51, "Boenen": 54, "Dortmund": 36, "Gelsenkirchen": 66, "Hagen": 20, "Iserlohn": 0, "Neuss": 85, "Schwerte": 15},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843, "Bochum": 56, "Boenen": 102, "Dortmund": 75, "Gelsenkirchen": 62, "Hagen": 66, "Iserlohn": 85, "Neuss": 0, "Schwerte": 75},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901, "Bochum": 39, "Boenen": 32, "Dortmund": 15, "Gelsenkirchen": 54, "Hagen": 14, "Iserlohn": 15, "Neuss": 75, "Schwerte": 0}
}

#Vehicle capacity
Q = 25
# Cost per km
C = 3

model3 = pe.ConcreteModel()

# Initialize sets for model 2
model3.F = pe.Set(initialize=factories)
model3.R = pe.Set(initialize=rebars.keys())
model3.L = pe.Set(initialize=longbars.keys())
model3.A = pe.Set(initialize=customers)
model3.T = pe.Set(initialize=periods)
model3.F_A = model3.F | model3.A
model3.A_f = pe.Set(model3.F, initialize=lambda model3, f: model3.A | {f})
#model3.C = pe.Set(model3.A, model3.T, within=model3.A, initialize=lambda model, a, t: [])


valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model3.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model3.x = pe.Var(model3.R, model3.L_N_F,  model3.T, domain=pe.NonNegativeIntegers)
model3.y = pe.Var(model3.L_N_F, model3.T, domain=pe.Binary)
model3.z = pe.Var(model3.F, model3.A, model3.T, domain=pe.Binary, initialize = 0)
model3.i = pe.Var(model3.A, model3.R, model3.T, domain=pe.NonNegativeIntegers)
model3.q = pe.Var(model3.R, model3.F, model3.A, model3.T, domain=pe.NonNegativeIntegers, initialize = 0)

# Define the valid indices for model3.v dynamically
#valid_v_indices = [(f, A_f, A_f, t) for f in model3.F for t in model3.T for A_f in model3.A_f[f]]
valid_v_indices = [
    (f, A_f_source, A_f_dest, t)
    for f in model3.F
    for t in model3.T
    for A_f_source in model3.A_f[f]
    for A_f_dest in model3.A_f[f]
]

# Add new variables to model3
model3.v = pe.Var(model3.F_A, model3.F_A, model3.T, domain=pe.Binary, initialize=0)
#model3.v = pe.Var(valid_v_indices, domain=pe.Binary, initialize = 0)
model3.u = pe.Var(model3.A, model3.F, model3.T, domain=pe.NonNegativeIntegers, initialize = 0)

# Add notation for t_fat and z_fat for simplicity
# Define t_fat as the tonnes shipped from factory f to customer a in period t
model3.t = pe.Expression(model3.F, model3.A, model3.T, 
                                  rule=lambda model3, f, a, t: 
                                  sum(model3.q[(r, f, a, t)] * w_r[r] * density * area for r in model3.R))
      
#model3.z = pe.Expression(model3.F, model3.A, model3.T, rule=lambda model3, f, a, t: sum(model3.v[(i, a, t)] for i in model3.A_f[f]))
  

# Constraints 
# 1. Cutting constraints - same as Q1
model3.cutting_cnstr = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for l in model3.L:
            for n in possible_longbars[l][f]:
                expression = sum(model3.x[r, l, n, f, t] * w_r[r] for r in model3.R) <= model3.y[l, n, f, t] * w_l[l]
                model3.cutting_cnstr.add(expression)

# 2. Production capacity - same as Q1
model3.capacity_cnstr = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model3.L) * density * area <= production_capacity[f]
        model3.capacity_cnstr.add(expression)

# Factory output - same as Q2
model3.factory_output = pe.ConstraintList()
for f in model3.F:
    for r in model3.R:
        for t in model3.T:
            expression = sum(model3.q[(r, f, a, t)] for a in model3.A) == sum(model3.x[(r, l, n, f, t)] for n in possible_longbars[l][f] for l in model3.L)
            model3.factory_output.add(expression)

# Link z to q - same as Q2
model3.enforce_factory_usage = pe.ConstraintList()
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            expression = sum(model3.q[(r, f, a, t)] for r in model3.R) <= M * model3.z[f, a, t]
            model3.enforce_factory_usage.add(expression)
            
#  production triggers customer assignment constraint (link y to z) - same as Q1
model3.prod_triggers_customer_assignment = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        # Sum over all r, l, n as specified
        expression = sum(model3.y[ l, n, f, t]
                         for l in model3.L 
                         for n in possible_longbars[l][f]) <= M * sum(model3.z[f, a, t] for a in model3.A)
        
        # Add the constraint to the model
        model3.prod_triggers_customer_assignment.add(expression)


# Demand - same as Q2
model3.demand = pe.ConstraintList()
for a in model3.A:
    for r in model3.R:
        for t in model3.T:
            shipped = sum(model3.q[(r, f, a, t)] for f in model3.F) 
            if t == 1:
                expression = shipped - model3.i[(a,r,t)] == demand[r][a][t-1] 
                model3.demand.add(expression)
            else:
                expression = shipped - model3.i[(a,r,t)] + model3.i[(a,r,t-1)] == demand[r][a][t-1] 
                model3.demand.add(expression)

# Single sourcing - factory assignment - same as Q2
model3.single_sourcing_cnstr = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.z[(f,a,t)] for f in model3.F) <= 1
        model3.single_sourcing_cnstr.add(expression)

# Inventory - same as Q2
model3.inventory = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.i[(a,r,t)]* w_r[r] for r in model3.R) * density * area <= inventory_capacity[a]
        model3.inventory.add(expression)

# New constraints
                              
# Flow conservation
model3.flow_conservation = pe.ConstraintList()
for j in model3.F_A:
    for t in model3.T:
        expression = sum(model3.v[(i,j,t)] for i in model3.F_A) == sum(model3.v[(j,h,t)] for h in model3.F_A)
        model3.flow_conservation.add(expression)

# Subtour Elimination Constraint
model3.subtour_elimination = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A: 
            for j in model3.A:
                if i != j:
                    expression = model3.u[i,f,t] + model3.t[f,j,t] <= model3.u[j,f,t] + Q * (1 - model3.v[(i, j, t)])
                    model3.subtour_elimination.add(expression)

# Limiting t
model3.upper_bound_t = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A:
            expression = model3.t[f,i,t] <= model3.u[i,f,t]
            model3.upper_bound_t.add(expression)

# don't go from i to i
model3.prevent_loops = pe.ConstraintList()
for t in model3.T:
    for i in model3.F_A:
        expression = model3.v[(i, i, t)] == 0
        model3.prevent_loops.add(expression)


# Limiting u
model3.upper_bound_u = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A:
            expression = model3.u[i,f,t] <= Q
            model3.upper_bound_u.add(expression)

# Vehicle capacity
model3.vehicle_capacity = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.t[f,a,t] for a in model3.A) <= Q
        model3.vehicle_capacity.add(expression)


# don't go from one factory to another
model3.disconnect_factories = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.v[f,n,t] for n in model3.F) == 0
        model3.disconnect_factories.add(expression)

# ensure if theres shipment, v is positive
model3.ensure_shipment = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.t[(f, a, t)] for f in model3.F) <= M* sum(model3.v[i,a,t] for i in model3.F_A) 
        model3.ensure_shipment.add(expression)

# ensure if x>0, v is positive
model3.link_x_v = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.x[r, l, n, f, t] for f in model3.F for l in model3.L for r in model3.R for n in possible_longbars[l][f] ) <= M* sum(model3.v[i,a,t] for i in model3.F_A) 
        model3.link_x_v.add(expression)

# ensure if y>0, v is positive
model3.link_y_v = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.y[ l, n, f, t] for f in model3.F for l in model3.L for n in possible_longbars[l][f] ) <= M* sum(model3.v[i,a,t] for i in model3.F_A) 
        model3.link_y_v.add(expression)

# leave factory at most once
model3.leave_f_once = pe.ConstraintList()
for j in model3.F_A:
    for t in model3.T:
        expression = sum(model3.v[i,j,t] for i in model3.F_A ) <= 1
        model3.leave_f_once.add(expression)


# Objective function
objExpr = sum( C * sum(model3.v[(i, j, t)] * distanceq3.get(i, {}).get(j, 0) for i in model3.F_A) for j in model3.F_A for t in model3.T)
model3.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result3 = solver.solve(model3, tee=True, options={'TimeLimit':360})

print(f"\nSolver Status: {result3.solver.status}")
print(f"Termination Condition: {result3.solver.termination_condition}")
print(f"Objective value (Total Cost): {pe.value(model3.obj):.2f} €")

#debugging
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            for r in model3.R:
                if pe.value(model3.q[r, f, a, t]) > 0:
                    print(f"Rebar {r} is shipped from {f} to {a} in period {t} with quantity {pe.value(model3.q[r, f, a, t])}")

for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            if pe.value(model3.z[f, a, t]) > 0:
                print(f"Factory {f} is assigned to customer {a} in period {t}")


Set parameter Username
Set parameter LicenseID to value 2618603
Academic license - for non-commercial use only - expires 2026-02-05
Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpy13gkxt1.pyomo.lp
Reading time = 0.09 seconds
x1: 3384 rows, 8868 columns, 87916 nonzeros
Set parameter TimeLimit to value 360
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  360

Optimize a model with 3384 rows, 8868 columns and 87916 nonzeros
Model fingerprint: 0x84a5ad4e
Variable types: 0 continuous, 8868 integer (2532 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+02]
  Objective range  [4e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]


In [41]:
# Print all values of v (binary decision variable)
for t in model3.T:
    for i in model3.F_A:
        for j in model3.F_A:
        
            v_value = pe.value(model3.v[( i, j, t)])
            if v_value > 0:
                print(f"v[ {i}, {j}, {t}] = {v_value}")


v[ Segal, Neuss, 1] = 1.0
v[ Bochum, Dortmund, 1] = 1.0
v[ Boenen, Segal, 1] = 1.0
v[ Dortmund, Hagen, 1] = 1.0
v[ Gelsenkirchen, Bochum, 1] = 1.0
v[ Hagen, Iserlohn, 1] = 1.0
v[ Iserlohn, Schwerte, 1] = 1.0
v[ Neuss, Gelsenkirchen, 1] = 1.0
v[ Schwerte, Boenen, 1] = 1.0
v[ IJmuiden, Bochum, 2] = 1.0
v[ Segal, Bochum, 2] = 1.0
v[ South Wales, Bochum, 2] = 1.0
v[ Bochum, Dortmund, 2] = 1.0
v[ Bochum, Gelsenkirchen, 2] = 1.0
v[ Bochum, Neuss, 2] = 1.0
v[ Boenen, Segal, 2] = 1.0
v[ Boenen, South Wales, 2] = 1.0
v[ Dortmund, Schwerte, 2] = 1.0
v[ Gelsenkirchen, Boenen, 2] = 1.0
v[ Hagen, Iserlohn, 2] = 1.0
v[ Iserlohn, Boenen, 2] = 1.0
v[ Neuss, IJmuiden, 2] = 1.0
v[ Schwerte, Hagen, 2] = 1.0
v[ IJmuiden, Schwerte, 3] = 1.0
v[ Segal, Dortmund, 3] = 1.0
v[ South Wales, Gelsenkirchen, 3] = 1.0
v[ Bochum, Neuss, 3] = 1.0
v[ Boenen, IJmuiden, 3] = 1.0
v[ Dortmund, Boenen, 3] = 1.0
v[ Gelsenkirchen, Bochum, 3] = 1.0
v[ Hagen, Segal, 3] = 1.0
v[ Iserlohn, South Wales, 3] = 1.0
v[ Neuss, Schwerte

In [46]:
for t in model3.T:
    for f in model3.F:
        print(f"\nFactory {f} - Period {t}:")
        visited_customers = []

        # Identify the customers visited by this factory in period t
        for i in model3.A_f[f]:
            if pe.value(model3.v[(f, i, t)]) > 0.5:
                next_customer = i
                visited_customers.append(i)
                visited_set = {i}  # To track visited nodes

                while next_customer != f:
                    found_next = False  # Flag to check if we find a next customer

                    for j in model3.A_f[f]:
                        if (
                            i != j 
                            and pe.value(model3.v[next_customer, j, t]) > 0.5  # Safe access
                            and j not in visited_set  # Prevent cycling
                        ):
                            next_customer = j
                            visited_customers.append(next_customer)
                            visited_set.add(next_customer)
                            found_next = True
                            break  # Move to next customer

                    if not found_next:
                        break  # Stop if no valid next customer is found

        # Print the route
        if visited_customers:
            route_str = f"Start -> " + " -> ".join(map(str, visited_customers)) + " -> End"
            print(route_str)
        else:
            print("No route assigned (no customers visited).")



Factory IJmuiden - Period 1:
No route assigned (no customers visited).

Factory Segal - Period 1:
Start -> Neuss -> Gelsenkirchen -> Bochum -> Dortmund -> Boenen -> Schwerte -> Iserlohn -> Hagen -> Segal -> End

Factory South Wales - Period 1:
No route assigned (no customers visited).

Factory IJmuiden - Period 2:
Start -> Dortmund -> Schwerte -> Hagen -> Iserlohn -> Boenen -> IJmuiden -> End

Factory Segal - Period 2:
Start -> Bochum -> Gelsenkirchen -> Neuss -> Segal -> End

Factory South Wales - Period 2:
No route assigned (no customers visited).

Factory IJmuiden - Period 3:
Start -> Gelsenkirchen -> Bochum -> Neuss -> End

Factory Segal - Period 3:
No route assigned (no customers visited).

Factory South Wales - Period 3:
Start -> Schwerte -> Iserlohn -> Hagen -> Dortmund -> Boenen -> End

Factory IJmuiden - Period 4:
Start -> Bochum -> Neuss -> Gelsenkirchen -> IJmuiden -> End

Factory Segal - Period 4:
Start -> Schwerte -> Iserlohn -> Hagen -> Dortmund -> Boenen -> Segal -> End

In [30]:
# Analysis of production and demand

total_demand_per_period = {1: 0, 2: 0, 3: 0, 4: 0}
for r in rebars:
    for customer in demand[r]:
        for t in range(4):  # For each period
            # Demand in tonnes for each rebar type, customer, and period
            demand_in_tonnes = demand[r][customer][t] * w_r[r] * density * area
            total_demand_per_period[t + 1] += demand_in_tonnes

# Initialize a dictionary to store production data
factory_production = {f: {t: 0 for t in range(1, 5)} for f in ["IJmuiden", "Segal", "South Wales"]}
total_production_per_period = {t: 0 for t in range(1, 5)}  # Total production for each period

# Extract production values from model3
for f in factory_production:
    for t in factory_production[f]:
        total_tonnes = sum(
            pe.value(model3.y[l, n, f, t]) * w_l[l] * density * area
            for l in model3.L for n in possible_longbars[l][f]
            if pe.value(model3.y[l, n, f, t]) > 0.5
        )
        factory_production[f][t] = round(total_tonnes, 2)  # Round to 2 decimals
        total_production_per_period[t] += total_tonnes  # Update total production per period

# Print table
print("Factory Production (Tonnes per Period)")
print("-" * 80)
print(f"{'Factory':<15} {'P1':<10} {'P2':<10} {'P3':<10} {'P4':<10} ")
print("-" * 80)

# Print the factory production values
for f, periods in factory_production.items():
    row = f"{f:<15}"
    for t in range(1, 5):
        row += f"{periods[t]:<10}"
    print(row)

# Print total production across all factories
row = f"{'Total Production':<15}"
for t in range(1, 5):
    row += f"{total_production_per_period[t]:<10}"
row += f"{round(sum(total_production_per_period.values()), 2):<15.2f}"  # Rounded total production
print(row)

# Print total demand per period
row = f"{'Total Demand':<15}"
for t in total_demand_per_period:
    row += f"{round(total_demand_per_period[t], 2):<10}"
print(row)

print()

# Cost per period

total_cost_per_period = {t: 0 for t in model3.T}  # Dictionary to store total cost for each period

# Loop through each period, customer, and factory to calculate the total cost per period
for t in model3.T:
    period_cost = 0  # Initialize the cost for the current period
    for f in model3.F:
        for a in model3.A:
            # Calculate the cost of transportation and fixed costs for each factory, customer, and period
            
            period_cost += pe.value(model3.z[f, a, t]) * (
            C * (sum(pe.value(model3.v[(i, a, t)]) * distanceq3.get(i, {}).get(a, 0) for i in model3.A_f[f]) + pe.value(model3.v[(a, f, t)]) * distanceq3.get(a, {}).get(f, 0) )
            )
    # Store the total cost for this period
    total_cost_per_period[t] = period_cost

# Print the total cost per period
for t in total_cost_per_period:
    print(f"Total cost for Period {t}: €{total_cost_per_period[t]:.2f}")


Factory Production (Tonnes per Period)
--------------------------------------------------------------------------------
Factory         P1         P2         P3         P4         
--------------------------------------------------------------------------------
IJmuiden       10.52     11.96     11.96     10.22     
Segal          4.81      9.98      9.55      7.75      
South Wales    24.16     26.02     22.96     18.93     
Total Production39.4817026834146947.9549448118187544.46949769517027636.89766430383047168.80         
Total Demand   15.48     23.73     23.06     25.02     

Total cost for Period 1: €777.00
Total cost for Period 2: €1008.00
Total cost for Period 3: €372.00
Total cost for Period 4: €303.00


In [43]:
print("Non-null variable values:")

for var in model3.component_objects(pe.Var, active=True):  # Iterate over all variables
    print(f"\nVariable: {var.name}")
    for index in var:
        value = pe.value(var[index])
        if value is not None and value > 0.5:
            print(f"  {index} = {value}")


Non-null variable values:

Variable: x
  ('A', 1, 1, 'IJmuiden', 1) = 3.0
  ('A', 1, 3, 'IJmuiden', 1) = 3.0
  ('A', 1, 5, 'IJmuiden', 4) = 1.0
  ('A', 1, 10, 'IJmuiden', 1) = 1.0
  ('A', 1, 15, 'IJmuiden', 3) = 3.0
  ('A', 1, 28, 'IJmuiden', 1) = 3.0
  ('A', 1, 30, 'IJmuiden', 1) = 3.0
  ('A', 1, 31, 'IJmuiden', 2) = 2.0
  ('A', 1, 33, 'IJmuiden', 3) = 3.0
  ('A', 1, 34, 'IJmuiden', 4) = 3.0
  ('A', 1, 36, 'IJmuiden', 1) = 3.0
  ('A', 1, 39, 'IJmuiden', 3) = 2.0
  ('A', 1, 42, 'IJmuiden', 3) = 3.0
  ('A', 1, 42, 'IJmuiden', 4) = 3.0
  ('A', 1, 48, 'IJmuiden', 1) = 3.0
  ('A', 1, 6, 'Segal', 2) = 1.0
  ('A', 1, 17, 'Segal', 1) = 3.0
  ('A', 1, 18, 'Segal', 3) = 3.0
  ('A', 1, 20, 'Segal', 2) = 3.0
  ('A', 1, 24, 'Segal', 4) = 2.0
  ('A', 1, 33, 'Segal', 3) = 3.0
  ('A', 1, 34, 'Segal', 1) = 3.0
  ('A', 1, 38, 'Segal', 2) = 3.0
  ('A', 1, 39, 'Segal', 2) = 2.0
  ('A', 1, 39, 'Segal', 4) = 3.0
  ('A', 1, 2, 'South Wales', 1) = 3.0
  ('A', 1, 4, 'South Wales', 3) = 1.0
  ('A', 1, 9, 'Sout

In [47]:
print(f"Objective Function Expression: {model3.obj.expr}")


Objective Function Expression: 3*z[IJmuiden,Bochum,1]*(250*v[IJmuiden,Bochum,1] + 203*v[Segal,Bochum,1] + 866*v[South Wales,Bochum,1] + 0*v[Bochum,Bochum,1] + 55*v[Boenen,Bochum,1] + 21*v[Dortmund,Bochum,1] + 19*v[Gelsenkirchen,Bochum,1] + 41*v[Hagen,Bochum,1] + 51*v[Iserlohn,Bochum,1] + 56*v[Neuss,Bochum,1] + 39*v[Schwerte,Bochum,1] + 250*v[Bochum,IJmuiden,1]) + 3*z[IJmuiden,Bochum,2]*(250*v[IJmuiden,Bochum,2] + 203*v[Segal,Bochum,2] + 866*v[South Wales,Bochum,2] + 0*v[Bochum,Bochum,2] + 55*v[Boenen,Bochum,2] + 21*v[Dortmund,Bochum,2] + 19*v[Gelsenkirchen,Bochum,2] + 41*v[Hagen,Bochum,2] + 51*v[Iserlohn,Bochum,2] + 56*v[Neuss,Bochum,2] + 39*v[Schwerte,Bochum,2] + 250*v[Bochum,IJmuiden,2]) + 3*z[IJmuiden,Bochum,3]*(250*v[IJmuiden,Bochum,3] + 203*v[Segal,Bochum,3] + 866*v[South Wales,Bochum,3] + 0*v[Bochum,Bochum,3] + 55*v[Boenen,Bochum,3] + 21*v[Dortmund,Bochum,3] + 19*v[Gelsenkirchen,Bochum,3] + 41*v[Hagen,Bochum,3] + 51*v[Iserlohn,Bochum,3] + 56*v[Neuss,Bochum,3] + 39*v[Schwerte,Boch